### Nonlinear Problems in Finance (MATH-GA.2045-001), New York University, Fall 2019
# Homework II

### Due Date: 11:55 PM Tuesday, Oct 22 2019


You should turn in the notebook at NYU Classes website.

Please comment your code properly.

Before you turn in the notebook, press the "Run all cells" button in the toolbar, and make sure all the calculation results and graphs are produced correctly in a reasonable time frame, and then save the notebook.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

# Dual Problem and Upper Bounds for American Options

<strong>Primal Problem</strong> (maximizing over stopping times)

\begin{equation*}
V_t=\sup_{\tau\in\mathcal{T}_{t, T}}\mathbb{E}^\mathbb{Q}\left[D_{t, \tau}F_{\tau}\big|\mathcal{F}_t\right]
\end{equation*}

<strong>Dual Problem</strong> (minimizing over martingales) [Rogers 2002; Haugh and Kogan 2004]

\begin{equation*}
V_t=\inf_{M\in\mathcal{M}_{t, 0}}\mathbb{E}^\mathbb{Q}\left[\sup_{t\leq s\leq T}\left(D_{t, s}F_s-M_s\right)\bigg|\mathcal{F}_t\right].
\end{equation*}

where $\mathcal{M}_{t, 0}$ denotes the set of all right-continuous martingales $(M_s, s\in[t, T])$ with $M_t=0$.
The optimal martingale $M^{\ast}$ is the martingale part of the Doob-Meyer decomposition of $(S_s-S_t)/D_{0, t}$, $t\leq s\leq T$, where $S$ is the Snell envelope of the discounted payoff, i.e., $S_t=\sup_{\tau\in\mathcal{T}_{t,T}}\mathbb{E}^{\mathbb{Q}}\left[D_{0,\tau}F_{\tau}\right]$.

In particular, any martingale $M_s$ with $M_0=0$ gives an upper bound for the price at time 0, $V_0$:

$$V_0 \leq\mathbb{E}^{\mathbb{Q}}\left[\sup_{0\leq s\leq T}\left(D_sF_s-M_s\right)\right].$$

<b style="color:darkorange">Question 1.</b> Consider pricing a one-year Bermudan put option with monthly exercise, where the asset price process is assumed to follow geometric Brownian motion with $S_0=100$, $\sigma=0.2$, $r=0.1$, $q=0.02$, $K=100$ and exercise dates $t_1=\frac{1}{12}$, $t_2=\frac{2}{12}$, $\cdots$, $t_{12}=1$.

Simulate 100,000 paths and use the following martingale to find a upper bound.

(a). $M_t\equiv0$.

(b). $M_t$ is the discounted European put price with the same final maturity less the initial price.

For your reference, the price of the option is $5.152$.

In [2]:
def blackscholes_mc(S=100, vol=0.2, r=0, q=0, ts=np.linspace(0, 1, 13), npaths=10):
    """Generate Monte-Carlo paths in Black-Scholes model.

    Parameters
    ----------
    S: scalar
        The spot price of the underlying security.
    vol: scalar
        The implied Black-Scholes volatility.
    r: scalar
        The annualized risk-free interest rate, continuously compounded.
    q: scalar
        The annualized continuous dividend yield.
    ts: array_like
        The time steps of the simualtion
    npaths: int
        the number of paths to simulate

    Returns
    -------
    paths: ndarray
        The Monte-Carlo paths.
    """
    nsteps = len(ts) - 1
    ts = np.asfarray(ts)[:, np.newaxis]
    W = np.cumsum(np.vstack((np.zeros((1, npaths), dtype=np.float),
                             np.random.randn(nsteps, npaths) * np.sqrt(np.diff(ts, axis=0)))),
                  axis=0)
    paths = np.exp(-0.5*vol**2*ts + vol*W)*S*np.exp((r-q)*ts)
    return paths


def blackscholes_price(K, T, S, vol, r=0, q=0, callput='call'):
    """Compute the call/put option price in the Black-Scholes model
    
    Parameters
    ----------
    K: scalar or array_like
        The strike of the option.
    T: scalar or array_like
        The maturity of the option, expressed in years (e.g. 0.25 for 3-month and 2 for 2 years)
    S: scalar or array_like
        The current price of the underlying asset.
    vol: scalar or array_like
        The implied Black-Scholes volatility.
    r: scalar or array_like
        The annualized risk-free interest rate, continuously compounded.
    q: scalar or array_like
        The annualized continuous dividend yield.
    callput: str
        Must be either 'call' or 'put'.

    Returns
    -------
    price: scalar or array_like
        The price of the option.

    Examples
    --------
    >>> blackscholes_price(95, 0.25, 100, 0.2, r=0.05, callput='put')
    1.5342604771222823
    """
    F = S*np.exp((r-q)*T)
    v = np.sqrt(vol**2*T)
    d1 = np.log(F/K)/v + 0.5*v
    d2 = d1 - v
    try:
        opttype = {'call':1, 'put':-1}[callput.lower()]
    except:
        raise ValueError('The value of callput must be either "call" or "put".')
    price = opttype*(F*norm.cdf(opttype*d1)-K*norm.cdf(opttype*d2))*np.exp(-r*T)
    return price

### Answer of Question 1.a

In [3]:
# Setup parameters
S0 = 100
sig = 0.2
r = 0.1
q = 0.02
K = 100
ts = np.linspace(0, 1, 13)
npaths = 100000
T = 1

paths = blackscholes_mc(S0, sig, r, q, ts=ts, npaths=npaths)
payoff = np.maximum(K-paths[1:13,:], 0)
D = np.exp(-r * (ts[1:] - ts[0]))
np.mean(np.max(D.reshape((12,1))*payoff, axis = 0))

8.658930475529711

The result has a huge bias because 0 is not a good choice to hedge. It's a bad strategy.

### Answer of Question 1.b

In [4]:
paths = blackscholes_mc(S0, sig, r, q, ts=ts, npaths=npaths)
M = np.array([blackscholes_price(K, T - ts[i],paths[i] , sig, r, q, callput='put') for i in range(1,len(paths))])
M0 = np.array(blackscholes_price(K, T, paths[0] , sig, r, q, callput='put'))
payoff = np.maximum(K-paths[1:13,:], 0)
D = np.exp(-r * (ts[1:] - ts[0]))
np.mean(np.max(D.reshape((12,1))*(payoff - M) + M0, axis = 0))

c:\users\tiany\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in true_divide


5.354699862767054

This time, the result doesn't have such a large bias. Using the discounted European put price with the same final maturity less the initial price is obviously better than 0.

## Optimal martingale

The optimal martingale that gives zero <em>duality gap</em> is the martingale component of the discounted true value process (known as Snell envelope) $\frac{V_t}{D_{0,t}}$, following the optimal exercise strategy.

To get a good upper bound, we first find a value process that is close to the true value process, and then extract its martingale component. This approximate value process can be obtained from a functional approximation such as regression, or from an exercise strategy (stopping time).

In general, for a stochastic process $\{U_n\}_{n\geq0}$, to extract its martingale component, we only need to let $M_0=0$, and then

$$M_{n+1}-M_n=U_{n+1}-\mathbb{E}\left[U_{n+1}\big|\mathcal{F}_n\right],\quad n=0,1,\cdots.$$

As a demonstration, we show below the code that calculates the upper bound estimate of the price of Bermudan put with monthly exercises as in Question 1.1. The optimal exercise frontier is given for each exercise date (from a PDE solver), i.e. if the underlying price falls below Bs[i] at time ts[i], then we should exercise the option, otherwise we should continue.

In [5]:
Bs = np.array([ 87.52900166,  87.80954309,  88.09098369,  88.37332635, 88.72752759,  89.1544435 ,  89.61925406,  90.19465661, 90.88274744,  91.75942312,  92.97867688,  94.85697075, 100.])

def exer_or_cont(i, S):
    return S <= Bs[i]

In [6]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)
exer_func = lambda S: np.maximum(K-S, 0)

# nested simulation from time t_i when stock price is S
def nested_mc(S, vol, r, q, i, ts, nnested):
    nested_paths = np.full(nnested, S, dtype=np.float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        exer_vals =  exer_func(nested_paths)
        if j < len(ts)-1:
            ind = exer_or_cont(j, nested_paths)                    # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_paths = nested_paths[~ind]                      # remove exercised paths 
            if len(nested_paths) == 0:                             # if exercised for all paths, stop
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths

# Simulate independent paths and exercise the option acoording to this strategy
V0 = nested_mc(S0, vol, r, q, 0, ts, 1000000)

# Upper bound by Andersen-Broadie algorithm
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
V = np.full(paths.shape, np.nan, dtype=np.float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=np.float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    exer_vals =  exer_func(paths[i])          
    ind = exer_or_cont(i, paths[i])                                # True for exercise False for continue
    for j in range(npaths):
        if exer_or_cont(i, paths[i][j]):
            V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                     # if exercised, V[i,j] = exercise value
            EV[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested) # launch nested simulation to estimate E[V_{i+1}|F_i]
        else:
            V[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested)  # if continue, use nested simulation to estimate V[i, j]
            EV[i, j] = V[i, j]                                          # E[V_{i+1}|F_i] = V_i
V[-1] = exer_func(paths[-1])*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(paths.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martinglae increment V_{i+1}-E[V_{i+1}|F_i]

print('Primal Price (Lower Bound) = {:.4f}'.format(V0))
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

Primal Price (Lower Bound) = 5.1584
Dual Price   (Upper Bound) = 5.1656


Since the strategy is optimal, we see that both lower bound and upper bound prices converge to the true value. 

However, in general, we may only get a sub-optimal strategy (e.g. from Longstaff-Schwartz algorithm). But with this sub-optimal strategy, we can build the (discounted) value process and then extract its martingale component to obtain an upper bound. Intuitively speaking, the closer this sub-optimal strategy is to the optimal one, the tighter the corresponding upper bound.

<b style="color:darkorange">Question 2.</b> Consider pricing the Bermudan put option as in Question 1.1 by using the constant $1.0$ and the Black-Scholes price of a European put option with volatility $0.2$ and maturity $T-t_n$ as the two basis functions at time $t_n$. Use the Longstaff-Schwartz algorithm to build an exercise strategy with these basis functions and then compute the corresponding upper bound.

For your convenience, an implementation of Longstaff-Schwartz algorithm is included in the cell below. It computes the regression coefficients at each exercise date.

In [7]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

npaths = 10000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
betas_LS = np.full((len(ts), 2), np.nan, dtype=np.float)
payoff = np.maximum(K-paths[-1], 0)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas_LS[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]    # regression to estimate continuation values
    contval = betas_LS[i, 0]+betas_LS[i, 1]*Z
    exerval = np.maximum(K-paths[i], 0)
    # identify the paths where we should exercise
    ind = exerval > contval
    payoff[ind] = exerval[ind]                     # update payoff on exercised paths
betas_LS                                           # No regression needed at first and last time steps, return NaN

array([[        nan,         nan],
       [-0.33140168,  1.25667347],
       [-0.49315932,  1.30491894],
       [-0.40911939,  1.27997506],
       [-0.27359828,  1.22862932],
       [-0.16976728,  1.18796836],
       [-0.10605114,  1.15825712],
       [-0.01573348,  1.12857489],
       [ 0.03409057,  1.09161169],
       [ 0.04199472,  1.07890181],
       [ 0.06333976,  1.02160387],
       [ 0.0281511 ,  0.995943  ],
       [        nan,         nan]])

### Answer of Question 2

In [8]:
# Setup parameters
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)
npaths = 10000
T = 1
nnested = 1000

In [9]:
# The following code use the above given code as a reference

# nested simulation from time t_i when stock price is S
def nested_mc_longstaff(S, vol, r, q, i, ts, nnested):
    nested_paths = np.full(nnested, S, dtype=np.float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        exer_vals =  exer_func(nested_paths)
        if j < len(ts)-1:
            BS_price_nested = blackscholes_price(K, T - ts[j], nested_paths , vol, r, q, callput='put')
            cont_vals = betas_LS[j][0] +  betas_LS[j][1] * BS_price_nested
            ind = cont_vals < exer_vals                            # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_paths = nested_paths[~ind]                      # remove exercised paths 
            if len(nested_paths) == 0:                             # if exercised for all paths, stop
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths

# Simulate independent paths and exercise the option acoording to this strategy
V0 = nested_mc(S0, vol, r, q, 0, ts, 1000000)

# Upper bound by Andersen-Broadie algorithm
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
V = np.full(paths.shape, np.nan, dtype=np.float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=np.float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    exer_vals =  exer_func(paths[i])
    BS_price_paths = blackscholes_price(K, T - ts[i], paths[i] , vol, r, q, callput='put')
    cont_vals = betas_LS[i][0] +  betas_LS[i][1] * BS_price_paths
    ind = cont_vals < exer_vals                                    # True for exercise False for continue
    for j in range(npaths):
        if ind[j]:
            V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                     # if exercised, V[i,j] = exercise value
            EV[i,j] = nested_mc_longstaff(paths[i, j], vol, r, q, i, ts, nnested) # launch nested simulation to estimate E[V_{i+1}|F_i]
        else:
            V[i,j] = nested_mc_longstaff(paths[i, j], vol, r, q, i, ts, nnested)  # if continue, use nested simulation to estimate V[i, j]
            EV[i, j] = V[i, j]                                          # E[V_{i+1}|F_i] = V_i
V[-1] = exer_func(paths[-1])*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(paths.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martinglae increment V_{i+1}-E[V_{i+1}|F_i]
print('Primal Price (Lower Bound) = {:.4f}'.format(V0))
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))
#np.mean(np.amax(((exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis]))-hedges[1:]), axis=0))

Primal Price (Lower Bound) = 5.1527
Dual Price   (Upper Bound) = 5.1738


As we can see, this time, the result is much better than the result of Question 1 because we use the two right basis functions. Even if the strategy is sub-optimal given by Longstaff-Schwartz algorithm, we still see that both lower bound and upper bound prices converge to the true value.

<b style="color:darkorange">Question 3.</b> <b>(An alternative algorithm to estimate the upper bound)</b> The primal algorithm (Longstaff-Schwartz or Tsitsiklis-Van Roy) provides a function approximation (via regression) of the continuation value $C_n$ at $t_n$, hence of the value function $V_n = \max(C_n,F_n)$ where $F_n$ is the exercise value. We can use this functional approximation of the value function to write an alternative algorithm in order to estimate the upper bound: at each time $t_n$, use the functional approximation of $V_n$, and run the nested simulation to estimate the conditional expectation $\mathbb{E}\left[V_{n+1}\big|\mathcal{F}_n\right]$ (using the functional approximation of $V_{n+1}$). Note that this nested simulation only needs to be run for one time step.

For the same Bermudan put option and same basis functions, use TVR algorithm to build the value process and then extract its martingale component using the alternative algorithm explained above to estimate the upper bound price.

For your convenience, an implementation of TVR algorithm is included in the cell below. It computes the regression coefficients at each exercise date. 

In [10]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

npaths = 100000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
betas_TVR = np.full((len(ts), 2), np.nan, dtype=np.float)
V = np.maximum(K-paths[-1], 0)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas_TVR[i] = np.linalg.lstsq(A, V*discount, rcond=None)[0]      # regression to estimate continuation values
    contval = betas_TVR[i, 0]+betas_TVR[i, 1]*Z
    exerval = np.maximum(K-paths[i], 0)
    V = np.maximum(exerval, contval)                      # compute values
betas_TVR                                                 # No regression needed at first and last time steps, return NaN

array([[        nan,         nan],
       [-0.44519091,  1.31329769],
       [-0.37267303,  1.29195262],
       [-0.2734191 ,  1.26138541],
       [-0.1957102 ,  1.23072946],
       [-0.13617827,  1.20113657],
       [-0.07841976,  1.1665675 ],
       [-0.02517096,  1.13291785],
       [ 0.01739115,  1.09633981],
       [ 0.02937577,  1.06095475],
       [ 0.01332569,  1.02958688],
       [ 0.00724647,  0.99859021],
       [        nan,         nan]])

### Answer for the Question 3

In [11]:
# Setup parameters
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)
npaths = 10000
T = 1
nnested = 1000

In [12]:
# The following code use the above given code as a reference

# nested simulation from time t_i when stock price is S
def nested_mc_TVR(S, vol, r, q, i, ts, nnested):
    nested_paths = np.full(nnested, S, dtype=np.float)
    dt = ts[i+1] - ts[i]
    dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
    nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
    exer_vals =  exer_func(nested_paths)
    if i < len(ts) - 2:
        BS_price_nested = blackscholes_price(K, T - ts[i + 1], nested_paths , vol, r, q, callput='put')
        cont_vals = betas_LS[i + 1][0] +  betas_LS[i + 1][1] * BS_price_nested
        return np.mean(np.exp(-r * ts[i + 1]) * np.maximum(exer_vals, cont_vals))
    else:
        return np.mean(np.exp(-r * ts[i + 1]) * exer_vals)
    

# Simulate independent paths and exercise the option acoording to this strategy
V0 = nested_mc(S0, vol, r, q, 0, ts, 1000000)
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
V = np.full(paths.shape, np.nan, dtype=np.float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=np.float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    print("current time: ", i)
    exer_vals =  exer_func(paths[i])
    BS_price_paths = blackscholes_price(K, T - ts[i], paths[i] , vol, r, q, callput='put')
    cont_vals = betas_LS[i][0] +  betas_LS[i][1] * BS_price_paths
    V[i] = np.exp(-r * ts[i]) * np.maximum(exer_vals, cont_vals)
    EV[i] = [nested_mc_TVR(paths[i,j], vol, r, q, i, ts, nnested) for j in range(len(paths[i]))]
    
V[-1] = exer_func(paths[-1])*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(paths.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martinglae increment V_{i+1}-E[V_{i+1}|F_i]
#print(V[0,0])
#np.mean(np.amax(((exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis]))-hedges[1:]), axis=0))
#print('Primal Price (Lower Bound) = {:.4f}'.format(V[0,0])) uncomment this line if you want the lower bar which is not required in this question
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

current time:  1
current time:  2
current time:  3
current time:  4
current time:  5
current time:  6
current time:  7
current time:  8
current time:  9
current time:  10
current time:  11
Dual Price   (Upper Bound) = 5.1918


The TVR based method is not that good than the method we use in the Question 2. The error is larger than the Question 2 but it's still better than any result from the Question 1.

The upper bound estimator can be used to check if the basis functions used in regression are adequate. A tight duality gap between the lower bound and upper bound is a confirmation that the linear span of the selected basis functions indeed gives a good approximation to the continuation value. We now illustrate this with the pricing of Bermudan-Asian call options.

<b style="color:darkorange">Question 4</b> (<b>Bermudan-Asian Call Option</b>). In Question 2.5 of Homework I, we priced a Bermudan-Asian call option using a particular set of basis functions. Assume that all the parameters stay the same.

<b>(a).</b> Reuse the exercise strategy derived from the basis functions used in Question 2.5 of Homework I by Longstaff-Schwartz with $\bar{\sigma}=0.1$ to estimate the lower and upper bound. For your convenience, an implementation of Longstaff-Schwartz is included in the cell below. Note that the lower bound should be obtained by running new independent paths stopped according to the strategy from the Longstaff-Schwartz algorithm.

<b>(b).</b> As an alternative set of basis functions at time $t_n$, use 
$$1, A_{t_n}, S_{t_n}, A_{t_n}^2, S_{t_n}^2, A_{t_n}S_{t_n}.$$
in the Longstaff-Schwartz algorithm and then give the corresponding lower bound and upper bound.

<b>(c).</b> Compare the numerical duality gaps obtained from (a) and (b). Which set of basis functions does a better job at estimating the optimal exercise strategy?

### Answer for the Question 4.a

$$
Z_{t_n}\equiv \frac{nA_{t_n}+(12-n)S_{t_n}}{12}
$$
Just like what we have done in the previous homework, now we are using $Z_t$ as the basis function. Noted that Z is expected value of the Asian call at maturity.

In [13]:
# Longstaff-Schwartz
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
ts = np.linspace(0, 1, 13)
npaths = 100000
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths)                                  # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
vol_ = 0.1
payoff = np.maximum(pathsA[-1]-K, 0)
betas_AC = np.full((len(ts), 2), np.nan, dtype=np.float)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    Z = (i*pathsA[i]+(12-i)*pathsS[i])/12
    callZ = blackscholes_price(K, ts[-1]-ts[i], Z, vol_, callput='call')
    A = np.vstack((np.ones_like(Z), callZ)).T
    betas_AC[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]           # regression to estimate continuation values
    contval = betas_AC[i, 0]+betas_AC[i, 1]*callZ
    exerval = np.maximum(pathsA[i]-K, 0)
    ind = exerval > contval                               # identify the paths where we should exercise
    payoff[ind] = exerval[ind]
betas_AC # no need to estimate continuation values at the first date (not an exercise date) or the last date (always exercise)

array([[        nan,         nan],
       [ 0.19864601,  1.14684159],
       [-0.00123628,  1.11008422],
       [-0.17697248,  1.08994874],
       [-0.28566748,  1.07301417],
       [-0.34699774,  1.05744156],
       [-0.37364586,  1.04613112],
       [-0.38798188,  1.04010267],
       [-0.36241459,  1.03296198],
       [-0.32435852,  1.02721116],
       [-0.25924511,  1.018227  ],
       [-0.17040117,  1.00806531],
       [        nan,         nan]])

In [14]:
# Setup parameters
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
vol_longstaff = 0.1
ts = np.linspace(0, 1, 13)
npaths = 100000
T = 1
nnested = 1000
exer_func = lambda A: np.maximum(A-K, 0)

In [15]:
def nested_mc_with_average(S, vol, r, q, i, ts, nnested, avg_value):
    nested_paths = np.full(nnested, S, dtype=np.float)
    tot_payoff = 0
    nested_avg_paths = np.full(nnested, avg_value, dtype=np.float)
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        nested_avg_paths = (nested_paths + nested_avg_paths * (j-1)) / j
        exer_vals =  np.maximum(nested_avg_paths - K, 0)
        if j != len(ts)-1:
            Z_paths = (j*nested_avg_paths+(12-j)*nested_paths)/12
            BS_price_nested = blackscholes_price(K, ts[-1]-ts[j], Z_paths, vol_longstaff, callput='call')        
            cont_vals = betas_AC[j, 0]+betas_AC[j, 1]*BS_price_nested
            ind = cont_vals < exer_vals
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_paths = nested_paths[~ind]
            nested_avg_paths = nested_avg_paths[~ind]
            if len(nested_paths) == 0:                             
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths

V0 = nested_mc_with_average(S0, vol, r, q, 0, ts, 1000000,0) 

pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=1000)                             
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis]))
                                                 
V = np.full(pathsS.shape, np.nan, dtype=np.float)                   
EV = np.full(pathsS.shape, np.nan, dtype=np.float)                                                    
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
  

for i in range(1, len(ts)-1):
    exer_vals =  exer_func(pathsA[i])
    pathsZ = (i * pathsA[i] + (12 - i) * pathsS[i]) / 12
    BS_price_nested = blackscholes_price(K, ts[-1]-ts[i], pathsZ, vol_longstaff, callput='call')        
    cont_vals = betas_AC[i, 0]+betas_AC[i, 1]*BS_price_nested
    ind = cont_vals < exer_vals
    for j in range(len(ind)):
        if ind[j]:
            V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                    
            EV[i,j] = nested_mc_with_average(pathsS[i, j], vol, r, q, i, ts, nnested, pathsA[i,j]) 
        else:
            V[i,j] = nested_mc_with_average(pathsS[i, j], vol, r, q, i, ts, nnested, pathsA[i,j])  
            EV[i, j] = V[i, j]   


V[-1] = exer_func(pathsA[-1])*np.exp(-r*ts[-1])       
hedges = np.zeros(pathsS.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)               

print('Primal Price (Lower Bound) = {:.4f}'.format(EV[0,0]))
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(((exer_func(pathsA[1:])*np.exp(-r*ts[1:, np.newaxis]))-hedges[1:]), axis=0))))

Primal Price (Lower Bound) = 5.3046
Dual Price   (Upper Bound) = 5.3882


### Answer for the Question 4.b

In this question, we are using
$$1, A_{t_n}, S_{t_n}, A_{t_n}^2, S_{t_n}^2, A_{t_n}S_{t_n}.$$
as the basis functions.

In [16]:
# Longstaff-Schwartz
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
ts = np.linspace(0, 1, 13)
npaths = 100000
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths)                                  # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
vol_ = 0.1
payoff = np.maximum(pathsA[-1]-K, 0)
betas_AC = np.full((len(ts), 6), np.nan, dtype=np.float)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    #Z = (i*pathsA[i]+(12-i)*pathsS[i])/12
    #callZ = blackscholes_price(K, ts[-1]-ts[i], Z, vol_, callput='call')
    A = np.vstack((np.ones_like(pathsA[i]), pathsA[i], pathsS[i], \
                   np.power(pathsA[i],2), np.power(pathsS[i],2), pathsA[i] * pathsS[i])).T
    betas_AC[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]           # regression to estimate continuation values
    contval = betas_AC[i, 0]+betas_AC[i, 1]*pathsA[i] +betas_AC[i, 2]* pathsS[i] + \
    betas_AC[i, 3] * np.power(pathsA[i],2) + betas_AC[i, 4] * np.power(pathsS[i],2) + \
    betas_AC[i, 5] * pathsA[i]*pathsS[i]
    exerval = np.maximum(pathsA[i]-K, 0)
    ind = exerval > contval                               # identify the paths where we should exercise
    payoff[ind] = exerval[ind]
betas_AC # no need to estimate continuation values at the first date (not an exercise date) or the last date (always exercise)

array([[            nan,             nan,             nan,
                    nan,             nan,             nan],
       [ 1.58349717e+02, -1.82903313e+00, -1.82903313e+00,
         7.06132966e-03,  7.06132677e-03,  7.06132696e-03],
       [ 1.55475772e+02, -1.29385896e+00, -2.30351415e+00,
         1.66349591e-03,  8.16260415e-03,  1.09933767e-02],
       [ 1.53713365e+02, -1.95125184e+00, -1.60358585e+00,
         9.52452874e-03,  8.64641981e-03,  2.36304919e-03],
       [ 1.46212913e+02, -2.19274976e+00, -1.20346180e+00,
         1.19326300e-02,  7.25736758e-03,  4.81249196e-04],
       [ 1.39073935e+02, -2.25887708e+00, -9.83031901e-01,
         1.23894973e-02,  5.90080014e-03,  5.29015053e-04],
       [ 1.35376089e+02, -2.45037520e+00, -7.08929551e-01,
         1.41986241e-02,  5.01469948e-03, -8.70176703e-04],
       [ 1.30117489e+02, -2.52746264e+00, -5.18047625e-01,
         1.49236596e-02,  4.06216056e-03, -1.27025258e-03],
       [ 1.24948196e+02, -2.54165660e+00, -3.925

In [17]:
# Setup parameters
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
vol_longstaff = 0.1
ts = np.linspace(0, 1, 13)
npaths = 100000
T = 1
nnested = 1000
exer_func = lambda A: np.maximum(A-K, 0)

In [18]:
def nested_mc_with_average_b(S, vol, r, q, i, ts, nnested, avg_value):
    nested_paths = np.full(nnested, S, dtype=np.float)
    tot_payoff = 0
    nested_avg_paths = np.full(nnested, avg_value, dtype=np.float)
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        nested_avg_paths = (nested_paths + nested_avg_paths * (j-1)) / j
        exer_vals =  np.maximum(nested_avg_paths - K, 0)
        if j != len(ts)-1:
            #Z_paths = (j*nested_avg_paths+(12-j)*nested_paths)/12
            #BS_price_nested = blackscholes_price(K, ts[-1]-ts[j], Z_paths, vol_longstaff, callput='call')        
            cont_vals = betas_AC[j, 0]+betas_AC[j, 1] * nested_avg_paths +betas_AC[j, 2]* nested_paths + \
            betas_AC[j, 3] * np.power(nested_avg_paths,2) + betas_AC[j, 4] * np.power(nested_paths,2) + \
            betas_AC[j, 5] * nested_avg_paths * nested_paths
            ind = cont_vals < exer_vals
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_paths = nested_paths[~ind]
            nested_avg_paths = nested_avg_paths[~ind]
            if len(nested_paths) == 0:                             
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths

V0 = nested_mc_with_average_b(S0, vol, r, q, 0, ts, 1000000,0) 

pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=1000)                             
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis]))
                                                 
V = np.full(pathsS.shape, np.nan, dtype=np.float)                   
EV = np.full(pathsS.shape, np.nan, dtype=np.float)                                                    
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
  

for i in range(1, len(ts)-1):
    exer_vals =  exer_func(pathsA[i])      
    cont_vals = betas_AC[i, 0]+betas_AC[i, 1]*pathsA[i] +betas_AC[i, 2]* pathsS[i] + \
    betas_AC[i, 3] * np.power(pathsA[i],2) + betas_AC[i, 4] * np.power(pathsS[i],2) + \
    betas_AC[i, 5] * pathsA[i]*pathsS[i]
    ind = cont_vals < exer_vals
    for j in range(len(ind)):
        if ind[j]:
            V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                    
            EV[i,j] = nested_mc_with_average_b(pathsS[i, j], vol, r, q, i, ts, nnested, pathsA[i,j]) 
        else:
            V[i,j] = nested_mc_with_average_b(pathsS[i, j], vol, r, q, i, ts, nnested, pathsA[i,j])  
            EV[i, j] = V[i, j]   


V[-1] = exer_func(pathsA[-1])*np.exp(-r*ts[-1])       
hedges = np.zeros(pathsS.shape, dtype=np.float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)               

print('Primal Price (Lower Bound) = {:.4f}'.format(EV[0,0]))
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(((exer_func(pathsA[1:])*np.exp(-r*ts[1:, np.newaxis]))-hedges[1:]), axis=0))))

Primal Price (Lower Bound) = 5.0807
Dual Price   (Upper Bound) = 5.4227


### Answer for the Question 4.c
In 4.a, we use the BS functions $Z_t$ in HW1 (Noted that Z is expected value of the Asian call at maturity). We get a relatively low duality gap. Due to the different random seed, the duality gap fluctuate a little bit but it's usually around 0.082.   

In 4.b, we use the linear and quadratic combinations of $A_{t_n}$ and $S_{t_n}$ as the basis function. We get a relatively high duality gap. Due to the different random seed, the duality gap fluctuate a little bit but it's usually around 0.3. The upper bound estimation is similar to the upper bound estimation in 4.a. It also approaches to the low-bias estimation in the homework 1.  

In contrast, I will definitely say 4.a is better than 4.b becasue 4.a has a relatively low duality gap. Both of the dual method and the primal method gives accurate results. For, 4.b, although the upper bound is good enough, the lower bound is not. The $1, A_{t_n}, S_{t_n}, A_{t_n}^2, S_{t_n}^2, A_{t_n}S_{t_n}$ are performming well in the estimation of primal price.  

In conclusion, the result is not surprise to me. It matches my intuition. As Olivier pointed out, $Z_t$ is the expected value of the Asian call at maturity. Also, $Z_t$ avoids using the furture information. Because of $Z_t$ is a vague estimation of to the Bermudan-Asian Call Option, they are always sensitive during the regression. However, $1, A_{t_n}, S_{t_n}, A_{t_n}^2, S_{t_n}^2, A_{t_n}S_{t_n}$ is just a simple combination of related variables. They are not so powerful than the $Z_t$ in this pricing problem.

<b style="color:darkorange">Question 5</b>. (<b>Multiple Exercises</b>) Consider a chooser put option that allows its holder to exercise the option up to 3 times among 12 possible monthly exercise dates
$t_1=\frac{1}{12}, t_2=\frac{2}{12}, \cdots, t_{12}=1$.
If exercised at time $t_i$, the option holder is paid $\left(K-S_{t_i}\right)^+$ with $K=100$. No repetitive exercise at the same exercise date is allowed.
The underlying security price is assumed to follow the geometric Brownian motion

$$\frac{dS_t}{S_t}=(r-q)dt+\sigma dW_t$$

with $S_0=100$, $r=0.06$, $q=0.02$ and $\sigma=0.2$.

Note that the standard Bermudan put option can be viewed as a special chooser put option with only one exercise right. Modify the Longstaff-Schwartz algorithm for Bermudan option to price this chooser option.

<ul>
<li> For the sake of simplicity, you may use quadratic polynomial basis functions in the regression
<li> As explained in Question 2.3, the polynomial regression usually give poor global performance. To improve the accuracy, the regression should be done only in the region where option is in the money
<li> After you have obtained estimations of continuation values from the Longstaff-Schwartz algorithm, run an independent Monte Carlo simulation to obtain a low-biased price.
</ul>

### Answer for the Question 5

In [22]:
# Setup parameters
S0, K, vol, r, q = 100, 100, 0.2, 0.06, 0.02
ts = np.linspace(0, 1, 13)
npaths = 10000
T = 1
exer_func = lambda S: np.maximum(K-S, 0)

In [23]:
# initialize betas and values under different exerecise_time_left states
paths = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths)
payoff = exer_func(paths[-1])                 # Payoff at end time
V1 = np.vstack([paths[:12], payoff])
V2 = V1[:]
V3 = V1[:]
V = np.array([V1,V2,V3])
BETAs = [[[[] for n in range(12)] for i in range(3)] for j in range(2)]
def iter_value_calculation(V, paths, BETAs, n_exe, t):
    """
    This function is used to iteratively calculate the price of this chooser put option
    
    Inputs:
    V, 2 dimension list || V[i], i = 0,1,2 is the value path for at different exercise time left
    paths, 1 dimension list || the price paths of the underlying asset
    BETAs, 3 dimension list || BETAs[0] stroe the regression parameter of the cont part
                            || BETAs[1] stroe the regression parameter of the exer part
    n_exe, int || the exercise time left
    t, int || current time stamp
    
    Outputs:
    V, 2 dimension list || V[i], i = 0,1,2 is the value path for at different exercise time left
    """
    if n_exe == 0:
        return np.zeros(paths[0].shape)
    if t == 12:
        return exer_func(paths[-1])  
    # update the cont part
    cont_payoff = D*V[n_exe - 1][t+1]
    cont_polyval = cont_payoff[:]
    ind = paths[t] - K <0
    BETAs[0][n_exe - 1][t] = np.polyfit(paths[t,ind], cont_payoff[ind], deg=2)
    cont_polyval[ind] = np.polyval(BETAs[0][n_exe - 1][t], paths[t,ind])
    
    # update the exe part
    exer_payoff = D*iter_value_calculation(V, paths, BETAs, n_exe - 1, t + 1)
    exer_polyval = exer_payoff[:]
    # Do the regression when we still have chance to exercise
    if n_exe > 1:
        BETAs[1][n_exe - 1][t] = np.polyfit(paths[t,ind], exer_payoff[ind], deg=2)
        exer_polyval[ind] = exer_func(paths[t, ind]) + np.polyval(BETAs[1][n_exe - 1][t], paths[t,ind])
    V[n_exe - 1][t] = np.maximum(cont_polyval, exer_polyval)
    return V[n_exe - 1][t]
for t in range(len(ts)-2, 0, -1):
    D = np.exp(-r*(ts[t+1]-ts[t]))
    payoff_estimation = iter_value_calculation(V, paths, BETAs, 3, t)
Longstaff_estimation = payoff_estimation*np.exp(-r*ts[1])
print ("Longstaff Estimation without running independent MC: ", np.mean(Longstaff_estimation))

Longstaff Estimation without running independent MC:  17.826844037380397


In [24]:
npaths = 100000
def low_bias_MC_calculation(npaths):
    """
    This function used the independent MC to get the low biased pricing value.
    Please run the above function first, because we need the BETAs from the above.
    
    Input:
    npaths, int || the number of independent MC path we want to get the low-biased result
    
    Output:
    inde_MC_V, float || the pricing value of the low-biased method
    """
    # initialize settings
    inde_MC_V = 0
    inde_MC_path = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths)
    cont_val = np.zeros(len(inde_MC_path[0]))
    exerval = np.zeros(len(inde_MC_path[0]))
    n_exe_left = np.ones(len(inde_MC_path[0]))*3
    # iterate through time
    for t in range(len(ts)-1):
        if t == len(ts) - 2:
            payoff = exer_func(inde_MC_path[-1])
            ind = n_exe_left > 0
            inde_MC_V += np.sum(np.exp(-r*(ts[-1]-ts[0]))*payoff[ind])/npaths
            return inde_MC_V
        # update the exerval
        for exe_times in [1,2,3]:
            if exe_times == 1:
                ind = (n_exe_left==1)
                exerval[ind] = exer_func(inde_MC_path[t+1,ind])
            else:
                ind = (n_exe_left==exe_times)
                exerval[ind] = exer_func(inde_MC_path[t+1,ind]) + np.polyval(BETAs[1][exe_times-1][t+1], inde_MC_path[t+1,ind])
            # update the contval
            contval[ind] = np.polyval(BETAs[0][exe_times-1][t+1], inde_MC_path[t+1,ind])
        payoff = exer_func(inde_MC_path[t+1])
        ind = exerval > contval
        ind[n_exe_left == 0] = 0
        n_exe_left[ind] -=1
        inde_MC_V += np.sum(np.exp(-r*(ts[i]-ts[0]))*payoff[ind])/npaths
print ("The low-biased result after running independent MC: ", low_bias_MC_calculation(npaths))

The low-biased result after running independent MC:  16.820892243130746
